## **The frist several cells are for Google Colab only**
It is used to link to your folder on Google Drive; To run the following command, remove the hashtages at teh beginning of the command lines 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Change the directory to your folder on Google Drive

In [ ]:
cd '/content/drive/My Drive/Colab Notebooks/MKTG6010'

/content/drive/My Drive/Colab Notebooks/MKTG6010


In [ ]:
## display the directory
!pwd

/content/drive/My Drive/Colab Notebooks/MKTG6010


## **Starting from now, the codes are for either Jupyter or Google Colab**

In [ ]:
!pip install scikit-learn==1.1.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 46.7 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


## Part1 Topic modelling
- Read the tweets_sport.csv data into Jupyter
- Conduct topic modelling analysis 
  - Generate the key words for top 3 topics
  - Generate the maps for top 3 topics
  - Combining them with the tweets, interpret the results 

In [ ]:
import pandas as pd
import numpy as np

#read .csv data into pd 
data_all = pd.read_csv('Data/tweets_sport.csv');


text=data_all['text']

#Change the text to lower case
text = text.map(lambda text: text.lower())
#Convert data from pandas format to list values  
text=text.values.tolist()

#print the first 3 tweets
print(text[:3])


['"rt @dariofranchitti: this bathurst race has been an absolute belter!! love the aussie commentary. was hoping for better for my mate @jcourtâ€¦"', '"rt @wtf1couk: gp3 fans while you\'re waiting for the race to resume, flick over to the #bathurst1000. exciting final few laps."', '"rt @sally_fitz: incredible final laps of #bathurst1000 remarkable comeback from @jamiewhincup coming from the back of the grid @redbullraciâ€¦"']


In [ ]:
#list the first 5 observations 
data_all.head()

,id,text,event_number,player_dummy,team_dummy,weekday,month,day,time,user_screen_name,retweets,likes,comments
0,5.210000e+17,"""rt @dariofranchitti: this bathurst race has b...",11,0,0,7,10,12,7:28,007_dot,0.0,0.0,0.0
1,5.210000e+17,"""rt @wtf1couk: gp3 fans while you're waiting f...",11,0,0,7,10,12,7:28,007_dot,0.0,0.0,0.0
2,5.210000e+17,"""rt @sally_fitz: incredible final laps of #bat...",11,1,0,7,10,12,7:31,007cnt,0.0,0.0,0.0
3,5.210000e+17,"""rt @v8supercars: top 10 - mostert, moffat, pe...",11,0,0,7,10,12,9:04,007pixxx,0.0,0.0,0.0
4,5.210000e+17,"""#bathurst1000 i present to you, the fully st...",11,0,0,7,10,12,10:20,007pixxx,0.0,0.0,0.0


In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Conduct lemmatization for the words in the text
from nltk.stem import WordNetLemmatizer

tokens=[]
for sent in text:
    temp=[WordNetLemmatizer().lemmatize(word) for word in sent.split(" ")]
    tokens.append(temp)

In [ ]:
# Customized the stopwords
from sklearn.feature_extraction import text 
my_additional_stop_words = ["bathurst","adelaide","http","tas","tcm","darwin","coateshire","sandown","clipsal","sydney", "townsville"]
stop_words = text.ENGLISH_STOP_WORDS.union(my_additional_stop_words)

### Latent Dirichlet Allocation
For conducting the Latent Dirichlet Allocation algorithm, we need to 
1. Convert a collection of text documents to a matrix of token counts.
2. Build a Latent Dirichlet Allocation Model
  - load the model
  - fit_transform

In [ ]:
%%time
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Set how many topics we need
NUM_TOPICS = 10
data_all['tokens']=tokens
text_train = list(data_all['tokens'].apply(lambda x: ' '.join(x)))

# Convert a collection of text documents to a matrix of token counts.
## min_df: ignore terms that have a document frequency strictly lower than the given threshold
## max_df: ignore terms that have a document frequency strictly higher than the given threshold
## stop_words: ‘english’, list
## lowercase: Convert all characters to lowercase before tokenizing.
## token_pattern: Regular expression denoting what constitutes a “token”
vectoriser = CountVectorizer(min_df=5, max_df=0.9,
                             stop_words=stop_words,lowercase=True,
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')

data_vectorized= vectoriser.fit_transform(text_train)

# Build a Latent Dirichlet Allocation Model
lda_model = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, random_state=258, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)
 
#text = "mclaughlin: we can win front row qualifying only the start for volvo says young gun"
#x = lda_model.transform(vectorizer.transform([text]))[0]
#print(x, x.sum())

CPU times: user 15.9 s, sys: 63.1 ms, total: 15.9 s
Wall time: 20.9 s


In [ ]:
print(lda_Z.shape)

(13523, 10)


### Show the top 10 most frequent words in each topic

In [ ]:
def print_topics(model, vectoriser, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx+1))
        print([(vectoriser.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])
 
print("LDA Model:")
print_topics(lda_model, vectoriser)
print("=" * 20)

LDA Model:
Topic 1:
[('morris', 607.0090904416668), ('chaz', 579.8046379508434), ('mostert', 542.3464781221942), ('paul', 531.4341916057526), ('just', 513.6636136315058), ('supercars', 431.8760032778609), ('mountain', 361.80821075336416), ('wow', 301.0098232730656), ('king', 268.26003200918177), ('xwrcjhmg', 206.1980304156728)]
Topic 2:
[('amp', 413.73167949706755), ('like', 327.6511433162216), ('day', 290.8138715867198), ('race', 148.34486606794226), ('start', 146.49978746828526), ('believe', 143.1790926982608), ('car', 127.8189733117516), ('thanks', 117.68380685793879), ('home', 116.12669092790851), ('job', 115.75436904959923)]
Topic 3:
[('car', 536.1150617409814), ('supercars', 370.24481507630856), ('fuel', 323.79145578324295), ('year', 256.2866649449783), ('whincup', 225.08046770848148), ('sport', 197.91005371165315), ('make', 192.33190699572577), ('safety', 177.19981686529334), ('svg', 171.29447437734493), ('got', 149.81890134761593)]
Topic 4:
[('amazing', 331.4420169949792), ('wa

/usr/local/lib/python3.9/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
x = lda_model.transform(data_vectorized)
print(x[3])


[0.0083339  0.00833333 0.00833391 0.00833344 0.00833333 0.00833342
 0.00833356 0.92499832 0.00833345 0.00833333]


In [ ]:
print(data_all['text'][3])

"rt @v8supercars: top 10 - mostert, moffat, percat, w davison, whincup, winterbottom, t kelly, r kelly, coulthard and lowndes Â #bathurst1000â€¦"


#### Generate the topics for new data sets

A row represents a tweet and columns showed that tweet's belonged probability in each 10 topics

In [ ]:
import pandas as pd
import numpy as np

topics=pd.DataFrame(x)
topics.columns=['topic1','topic2','topic3','topic4','topic5','topic6','topic7','topic8','topic9','topic10']

topics['tweet']=data_all['text']
topics['month']=data_all['month']
topics['day']=data_all['day']
pd.DataFrame(topics).to_csv('Data/tweets_by_topics_all_new.csv',index=False)

In [ ]:
topics

,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,tweet,month,day
0,0.008333,0.008333,0.008333,0.008333,0.008333,0.924999,0.008333,0.008333,0.008334,0.008333,"""rt @dariofranchitti: this bathurst race has b...",10,12
1,0.009091,0.009093,0.009095,0.009091,0.009092,0.009095,0.827256,0.100000,0.009097,0.009091,"""rt @wtf1couk: gp3 fans while you're waiting f...",10,12
2,0.008333,0.008333,0.008334,0.008334,0.008333,0.008333,0.008333,0.175000,0.758332,0.008333,"""rt @sally_fitz: incredible final laps of #bat...",10,12
3,0.008334,0.008333,0.008334,0.008333,0.008333,0.008333,0.008334,0.924998,0.008333,0.008333,"""rt @v8supercars: top 10 - mostert, moffat, pe...",10,12
4,0.025000,0.025000,0.025000,0.275001,0.025000,0.025015,0.025000,0.524984,0.025000,0.025000,"""#bathurst1000 i present to you, the fully st...",10,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13518,0.016667,0.016667,0.016667,0.183333,0.016667,0.016670,0.683330,0.016667,0.016667,0.016667,"""oh, that is absolutely heartbreaking for shan...",10,12
13519,0.050000,0.050000,0.549994,0.050000,0.050000,0.050000,0.050000,0.050006,0.050000,0.050000,"""rt @off1cial_sport: clipsal 500 #adelaide htt...",3,1
13520,0.025004,0.274993,0.025000,0.525000,0.025000,0.025000,0.025000,0.025000,0.025003,0.025000,"""storm &amp; sunset #adelaide http://t.co/8gym...",10,26
13521,0.016667,0.183332,0.016667,0.016667,0.183334,0.016671,0.349997,0.016667,0.016667,0.183332,"""""you're a pack of arseholes"" still the greate...",10,12


### Generate Topic Map

Using pyLDAvis to generate topic map

In [ ]:
!pip install PyLDAVis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 32.0 MB/s eta 0:00:00


In [ ]:
import pyLDAvis.sklearn
 
pyLDAvis.enable_notebook()

# The parameters wee need,
## LDA model: lda_model
## vectorized model: data_vectorized
## matrix of token counts: vectoriser

panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectoriser, mds='tsne', sort_topics = False)
panel
 

/usr/local/lib/python3.9/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.9/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/usr/local/lib/python3.9/dist-packages/sklearn/manifold/_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
0      -42.735924  181.543686       1        1  11.031700
1      167.649323  -92.860535       2        1   9.629480
2       53.084011  -26.154953       3        1  10.436603
3     -162.364777  102.463577       4        1   7.635957
4      -65.971291 -110.475449       5        1   7.435529
5       69.860023  118.290375       6        1  10.584128
6      179.285782   49.837364       7        1  10.021588
7       47.968170 -171.933990       8        1  11.323717
8      -48.525990   34.548641       9        1  13.615063
9     -174.653717  -40.336769      10        1   8.286236, topic_info=         Term         Freq        Total Category  logprob  loglift
1672  mostert  1112.000000  1112.000000  Default  30.0000  30.0000
2899  whincup   907.000000   907.000000  Default  29.0000  29.0000
1669   morris   584.000000   584.000000  Default  28.0000  28.0000
430      chaz   767.000000   767.000000  Default  27.0000  27.0000
1426      lap   696.000000   696.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
1693    murah   112.637642   173.305382  Topic10  -4.0877   2.0597
776       end   101.947787   151.787463  Topic10  -4.1874   2.0926
788   enligne    68.710130    85.017487  Topic10  -4.5820   2.2776
2812  victory    80.541269   115.197672  Topic10  -4.4231   2.1327
1103      guy    69.685190    99.571819  Topic10  -4.5679   2.1337

[425 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
10        8  0.974309  abcgrandstand
16        4  0.985207     absolutely
22        8  0.980558         action
55        2  0.858298            ago
55        7  0.124843            ago
...     ...       ...            ...
2980      3  0.995430           year
2987      9  0.983078            yes
2992      5  0.966731          young
2998      9  0.990685        zealand
3006      2  0.984895          ztpjs

[438 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

**Result Interpretation **

......

### Part 2 Combining topic modelling with sentiment analysis
- Generate sentiment scores for all the tweets (using SentimentIntensityAnalyzer from NKTK as shown in tute 8)
- Calculate the sentiment score for the 2 largest topics for each event day.


## The following codes will be ONLY provided after students submission. 